In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from itertools import product, permutations
from multiprocessing import Pool
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import warnings
import random
import torch
import torch.optim as optim
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings("ignore")
np.random.seed(42)
scaler_minmax=MinMaxScaler()


c:\Users\thanh\OneDrive - Queensland University of Technology\dataPHD\pytorchhere.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('travel_full.csv').drop(columns=['Unnamed: 0'])
origin_data=  pd.read_csv('Data_STS.csv', sep='\t')

origin_data=origin_data.fillna(0)

In [3]:
# from itertools import product

# def grid_search(model, param_grid, n_iter=300):
#     best_loss = float('inf')
#     best_params = None
#     for params in product(*param_grid.values()):
#         params = dict(zip(param_grid.keys(), params))
#         mf = model(**params)
#         mf.factorize(iter=n_iter)
#         loss = mf.total_loss[-1]
#         if loss < best_loss:
#             best_loss = loss
#             best_params = params
#     print('Best parameters:', best_params)
#     print('Best loss:', best_loss)
#     return best_params


def grid_search(model, param_grid, test, filename='travel_grid_search_results.xlsx'):
    best_loss = float('inf')
    best_params = None
    results = []
    for params in product(*param_grid.values()):
        params = dict(zip(param_grid.keys(), params))
        mf = model(**params)
        mf.evaluation(test)
        loss = mf.total_loss[-1]
        results.append({**params, 'loss': loss})
        if loss < best_loss:
            best_loss = loss
            best_params = params
    print('Best parameters:', best_params)
    print('Best loss:', best_loss)
    df = pd.DataFrame(results)
    df.to_excel(filename, index=False)
    return best_params
# import pandas as pd
# from itertools import product
# from multiprocessing import Process, Queue, cpu_count

# def factorize_worker(model, params_list, n_iter, result_queue):
#     for params in params_list:
#         mf = model(**params)
#         mf.factorize(iter=n_iter)
#         loss = mf.total_loss[-1]
#         result_queue.put({**params, 'loss': loss})

# def grid_search(model, param_grid, n_iter=2, n_jobs=2, filename='grid_search_results.xlsx'):
#     if n_jobs == -1:
#         n_jobs = cpu_count()

#     best_loss = float('inf')
#     best_params = None
#     results = []

#     params_list = [dict(zip(param_grid.keys(), p)) for p in product(*param_grid.values())]
#     result_queue = Queue()

#     processes = []
#     chunk_size = (len(params_list) + n_jobs - 1) // n_jobs

#     for i in range(n_jobs):
#         start = i * chunk_size
#         end = min((i + 1) * chunk_size, len(params_list))
#         p = Process(target=factorize_worker, args=(model, params_list[start:end], n_iter, result_queue))
#         processes.append(p)

#     for p in processes:
#         p.start()

#     for _ in range(len(params_list)):
#         results.append(result_queue.get())

#     for p in processes:
#         p.join()

#     df = pd.DataFrame(results)
#     df.to_excel(filename, index=False)

#     best_params = df.loc[df['loss'].idxmin()].to_dict()
#     best_loss = best_params.pop('loss')

#     print('Best parameters:', best_params)
#     print('Best loss:', best_loss)

#     return best_params



In [4]:
def cal(df):

    
    # calculate total number of possible user-item interactions
    num_users = df[df.columns[0]].nunique()
    num_items = df[df.columns[1]].nunique()
    num_possible_interactions = num_users * num_items
    
    # calculate total number of actual user-item interactions
    num_actual_interactions = df.shape[0]
    
    # calculate sparsity of ratings
    sparsity = 1 - (num_actual_interactions / num_possible_interactions)
    
    print(sparsity)
cal(df)

0.9686870559159716


In [5]:
def count_nan(df):
    """
    Returns the percentage of NaN values in a pandas DataFrame.
    """
    total_cells = df.size
    nan_cells = df.isna().sum().sum()
    nan_percentage = (nan_cells / total_cells) * 100
    print(nan_percentage)
count_nan(df)

0.0


In [6]:
# neighbors=int(df['userid'].value_counts().mean())
# imputer = KNNImputer(n_neighbors=neighbors)
# imputed_df = pd.DataFrame(imputer.fit_transform(df),columns=['userid', 'itemid', 'rating', 'Time', 'Location', 'Companion'])

# for i in imputed_df.columns: 
#     imputed_df[i] = imputed_df[i].astype('int')
# imputed_df
df.columns

Index(['userID', 'itemID', 'gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3', 'rating'],
      dtype='object')

# Generating Contextual Coeficient

In [7]:
# Assuming your dataframe is called df
X = df[['userID', 'gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3']]
y = df['rating']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the LinearSVC model
svm = LinearSVC(random_state=42)
svm.fit(X_train_scaled, y_train)

# Get feature importances
importances = svm.coef_

# Print the feature importances
features = ['userID', 'gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3']
for i in range(importances.shape[0]):
    print(f'Class {i+1}:')
    for j in range(importances.shape[1]):
        print(f'  {features[j]}: {importances[i, j]}')

# Calculate the mean importance for each feature across all classes
mean_importances = np.mean(importances, axis=0)

# Create a dictionary to map feature names to mean importances
feature_importance_dict = {feature: importance for feature, importance in zip(features, mean_importances)}

# Replace the values in 'Time', 'Location', and 'Companion' columns with their respective feature importances
df_replaced =df.copy()
for feature in features:
    df_replaced[feature] = df_replaced[feature] * feature_importance_dict[feature]

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.tree import DecisionTreeClassifier

# X = imputed_df[['Time', 'Location', 'Companion']]
# y = imputed_df['rating']

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Standardize the features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Fit the AdaBoost model with a DecisionTreeClassifier as the base estimator
# base_estimator = DecisionTreeClassifier(max_depth=1, random_state=42)
# ada = AdaBoostClassifier(base_estimator=base_estimator, random_state=42)
# ada.fit(X_train_scaled, y_train)

# # Get feature importances
# importances = ada.feature_importances_

# # Print the feature importances
# features = ['Time', 'Location', 'Companion']
# for i, importance in enumerate(importances):
#     print(f'{features[i]}: {importance}')

X = df[['itemID', 'gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3']]
y = df['rating']
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the LinearSVC model
svm = LinearSVC(random_state=42)
svm.fit(X_train_scaled, y_train)

# Get feature importances
importances = svm.coef_

# Print the feature importances
features = ['itemID', 'gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3']
for i in range(importances.shape[0]):
    print(f'Class {i+1}:')
    for j in range(importances.shape[1]):
        print(f'  {features[j]}: {importances[i, j]}')

# Calculate the mean importance for each feature across all classes
mean_importances = np.mean(importances, axis=0)

# Create a dictionary to map feature names to mean importances
feature_importance_dict = {feature: importance for feature, importance in zip(features, mean_importances)}

# Replace the values in 'Time', 'Location', and 'Companion' columns with their respective feature importances
df_replaced_item =df.copy()
for feature in features:
    df_replaced_item[feature] = df_replaced_item[feature] * feature_importance_dict[feature]

Class 1:
  userID: 0.02884661846573517
  gender: 0.003748811687425532
  opennessToExperience: -0.055352408491916
  conscientiousness: -0.04829555751191756
  extraversion: 0.026083182133697518
  agreeableness: 0.009183346696952761
  emotionalStability: -0.00353426397902396
  distance: -0.047368426675396515
  timeAvailable: -0.010222539374455632
  temperature: -0.04131110773194485
  crowdedness: -0.008299592126735946
  knowledgeOfSurroundings: -0.02216760628387529
  season: -0.03461790820442387
  budget: 0.0052499439981564654
  daytime: 0.011517031553850973
  weather: -0.02038698536042554
  companion: -0.029770750549493467
  mood: 0.016799667799987726
  weekday: -0.05630799728282222
  travelGoal: 0.012882472230471618
  transport: -0.027840901330798376
  category1: -0.08238176611357301
  category2: -0.011605018295691898
  category3: 0.006645169667302016
Class 2:
  userID: 0.0017839902946475727
  gender: -0.013382979676117736
  opennessToExperience: -0.017755159090501394
  conscientiousnes

In [8]:
df_fix_user = df_replaced[['gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3']]
df

,userID,itemID,gender,opennessToExperience,conscientiousness,extraversion,agreeableness,emotionalStability,distance,timeAvailable,...,weather,companion,mood,weekday,travelGoal,transport,category1,category2,category3,rating
0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,3.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,19.0,26.0,4.0
1,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,...,3.0,3.0,2.0,1.0,2.0,1.0,1.0,14.0,26.0,1.0
2,2.0,2.0,1.0,4.0,2.0,2.0,5.0,4.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,15.0,26.0,1.0
3,2.0,2.0,1.0,4.0,2.0,1.0,3.0,2.0,1.0,1.0,...,1.0,1.0,2.0,1.0,3.0,2.0,1.0,18.0,26.0,1.0
4,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,...,3.0,3.0,1.0,1.0,2.0,1.0,1.0,14.0,26.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,324.0,53.0,2.0,4.0,7.0,2.0,3.0,4.0,1.0,1.0,...,1.0,2.0,2.0,1.0,1.0,1.0,4.0,19.0,26.0,4.0
2530,324.0,46.0,2.0,4.0,7.0,2.0,3.0,4.0,1.0,1.0,...,3.0,3.0,1.0,1.0,2.0,1.0,4.0,13.0,26.0,5.0
2531,324.0,43.0,2.0,4.0,7.0,2.0,3.0,4.0,1.0,2.0,...,3.0,3.0,1.0,1.0,1.0,2.0,4.0,14.0,26.0,3.0
2532,324.0,56.0,2.0,4.0,7.0,2.0,3.0,4.0,1.0,2.0,...,1.0,3.0,2.0,1.0,1.0,2.0,8.0,15.0,26.0,4.0


In [9]:
df_fix_user = df_replaced[['gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3']]
for i in ['gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3']:
    for j in range(len(df_fix_user[i])):
        if origin_data[i][j] ==0: 
            df_replaced[i][j]=0
df_fix_user = df_replaced_item[['gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3']]
for i in ['gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3']:
    for j in range(len(df_fix_user[i])):
        if origin_data[i][j] ==0: 
            df_replaced_item[i][j]=0      
for i in df_fix_user.columns: 
    prefix = "_user"
    # create a new column name by adding the prefix to the original column name
    new_col = i+prefix
    # create the new column by copying the original column data to the new column
    df[new_col] = df_replaced[i]
for i in df_fix_user.columns: 
    prefix = "_item"
    # create a new column name by adding the prefix to the original column name
    new_col = i+prefix
    # create the new column by copying the original column data to the new column
    df[new_col] = df_replaced_item[i]
try:
    df=df.drop(columns=['Unnamed: 0'])
except KeyError:
    pass


In [10]:
df.columns

Index(['userID', 'itemID', 'gender', 'opennessToExperience',
       'conscientiousness', 'extraversion', 'agreeableness',
       'emotionalStability', 'distance', 'timeAvailable', 'temperature',
       'crowdedness', 'knowledgeOfSurroundings', 'season', 'budget', 'daytime',
       'weather', 'companion', 'mood', 'weekday', 'travelGoal', 'transport',
       'category1', 'category2', 'category3', 'rating', 'gender_user',
       'opennessToExperience_user', 'conscientiousness_user',
       'extraversion_user', 'agreeableness_user', 'emotionalStability_user',
       'distance_user', 'timeAvailable_user', 'temperature_user',
       'crowdedness_user', 'knowledgeOfSurroundings_user', 'season_user',
       'budget_user', 'daytime_user', 'weather_user', 'companion_user',
       'mood_user', 'weekday_user', 'travelGoal_user', 'transport_user',
       'category1_user', 'category2_user', 'category3_user', 'gender_item',
       'opennessToExperience_item', 'conscientiousness_item',
       'extrave

# Get Bias

In [11]:
df_replaced.head(30)
global_mean = df_replaced["rating"].mean()
user_bias = df_replaced.groupby("userID")["rating"].mean() - global_mean
item_bias = df_replaced.groupby("itemID")["rating"].mean() - global_mean
print(np.min(user_bias))
print(np.max(user_bias))
print(np.mean(user_bias))
print("SUM: ",np.sum(user_bias))
print(np.min(item_bias))
print(np.max(item_bias))
print(np.mean(item_bias))
print("SUM: ",np.sum(item_bias))

-2.4715864246250985
1.5284135753749015
0.21361193194488473
SUM:  69.42387788208754
-2.4715864246250985
1.5284135753749015
0.16928395559480514
SUM:  42.15170494310648


In [12]:
rating_matrix_original= df[['userID','itemID','rating']].pivot_table(values='rating',index='userID',columns='itemID', fill_value=0).astype('int')
rating_matrix_original.values

array([[3, 2, 4, ..., 0, 0, 0],
       [1, 1, 2, ..., 0, 0, 0],
       [1, 0, 2, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Generate real Laplacian

In [13]:

def to_int(x):
    if pd.isna(x):
        return x
    return int(x)
rating_matrix=rating_matrix_original
# df_onehot = pd.get_dummies(df_2[['category_id', ' artist']])
# similarity_matrix = cosine_similarity(df_onehot)
# def calculate_L(similarity_matrix):
#     L = []

#     for i in range(len(similarity_matrix)):
#         a = 0
#         for j in range(len(similarity_matrix)):
#             a = similarity_matrix[i][j] * np.sum((similarity_matrix[i] - similarity_matrix[j]))
#         L.append(a)
#     return L
# df.columns

# Get Laplacian

In [14]:
neighbors=52

# Compute adjacency matrices based on similarity
# user_adj_matrix = adjacency_matrix_similarity(U_matrix)
# item_adj_matrix = adjacency_matrix_similarity(V_matrix)
# # Compute Laplacian matrices
# L_U = laplacian_matrix(user_adj_matrix)
# L_V = laplacian_matrix(item_adj_matrix)
# Get context matrix
scaler = StandardScaler()
# minmax = MinMaxScaler(feature_range=(0,2))
Cu=df[['gender_user', 'opennessToExperience_user', 'conscientiousness_user', 'extraversion_user', 'agreeableness_user', 'emotionalStability_user', 'distance_user', 'timeAvailable_user', 'temperature_user', 'crowdedness_user', 'knowledgeOfSurroundings_user', 'season_user', 'budget_user', 'daytime_user', 'weather_user', 'companion_user', 'mood_user', 'weekday_user', 'travelGoal_user', 'transport_user', 'category1_user', 'category2_user', 'category3_user',]]
Ci=df[['gender_item', 'opennessToExperience_item', 'conscientiousness_item', 'extraversion_item', 'agreeableness_item', 'emotionalStability_item', 'distance_item', 'timeAvailable_item', 'temperature_item', 'crowdedness_item', 'knowledgeOfSurroundings_item', 'season_item', 'budget_item', 'daytime_item', 'weather_item', 'companion_item', 'mood_item', 'weekday_item', 'travelGoal_item', 'transport_item', 'category1_item', 'category2_item', 'category3_item']]
C_umatrix =  np.array(Cu)
C_imatrix =  np.array(Ci)
print(np.min(C_imatrix))
print(np.max(C_imatrix))
print(np.median(C_imatrix))
print("SUM: ",np.sum(C_imatrix))
print(np.min(C_umatrix))
print(np.max(C_umatrix))
print(np.median(C_umatrix))
print("SUM: ",np.sum(C_umatrix))

-0.16699261018513675
0.0031041526938645476
0.0
SUM:  -48.91980299144071
-0.16382234926914035
0.0040578905157537875
0.0
SUM:  -50.66990928245317


In [15]:
def rmse(predicted_ratings, real_ratings):
    # Create a mask of the same shape as real_ratings with True where there's a rating and False where there's NaN
    mask = ~np.isnan(real_ratings)

    # Calculate the squared error between the predicted and real ratings only for the rated items
    squared_error = (predicted_ratings[mask] - real_ratings[mask])**2

    # Calculate the mean squared error
    mean_squared_error = np.mean(squared_error)

    # Calculate the root mean squared error
    root_mean_squared_error = np.sqrt(mean_squared_error)

    return root_mean_squared_error

def mae(predicted_ratings, real_ratings):
    # Create a mask of the same shape as real_ratings with True where there's a rating and False where there's NaN
    mask = ~np.isnan(real_ratings)

    # Calculate the absolute error between the predicted and real ratings only for the rated items
    absolute_error = np.abs(predicted_ratings[mask] - real_ratings[mask])

    # Calculate the mean absolute error
    mean_absolute_error = np.mean(absolute_error)

    return mean_absolute_error
def top_10_f1_score(predicted_ratings, real_ratings):
    f1_sum = 0
    users_count = real_ratings.shape[0]

    for user_idx in range(users_count):
        user_real_ratings = real_ratings[user_idx]
        user_predicted_ratings = predicted_ratings[user_idx]

        # Get the indices of the top-10 predicted ratings
        top_10_predicted_indices = np.argsort(user_predicted_ratings)[-10:]

        # Get the indices of the user's real ratings
        real_rated_indices = np.where(~np.isnan(user_real_ratings))[0]

        # Calculate the number of relevant items in the top-10 predicted items
        relevant_items_count = np.sum(np.isin(top_10_predicted_indices, real_rated_indices))

        # Calculate the precision for the current user
        user_precision = relevant_items_count / 10

        # Calculate the recall for the current user
        user_recall = relevant_items_count / len(real_rated_indices)

        # Calculate the F1-score for the current user
        if user_precision + user_recall > 0:
            user_f1_score = 2 * user_precision * user_recall / (user_precision + user_recall)
        else:
            user_f1_score = 0

        # Update the F1-score sum
        f1_sum += user_f1_score

    # Calculate the average F1-score across all users
    average_f1_score = f1_sum / users_count

    return average_f1_score
def top_10_precision(predicted_ratings, real_ratings):
    precision_sum = 0
    users_count = real_ratings.shape[0]

    for user_idx in range(users_count):
        user_real_ratings = real_ratings[user_idx]
        user_predicted_ratings = predicted_ratings[user_idx]

        # Get the indices of the top-10 predicted ratings
        top_10_predicted_indices = np.argsort(user_predicted_ratings)[-10:]

        # Get the indices of the user's real ratings
        real_rated_indices = np.where(~np.isnan(user_real_ratings))[0]

        # Calculate the number of relevant items in the top-10 predicted items
        relevant_items_count = np.sum(np.isin(top_10_predicted_indices, real_rated_indices))

        # Calculate the precision for the current user
        user_precision = relevant_items_count / 10

        # Update the precision sum
        precision_sum += user_precision

    # Calculate the average precision across all users
    average_precision = precision_sum / users_count

    return average_precision

def top_1_precision(predicted_ratings, real_ratings):
    precision_sum = 0
    users_count = real_ratings.shape[0]

    for user_idx in range(users_count):
        user_real_ratings = real_ratings[user_idx]
        user_predicted_ratings = predicted_ratings[user_idx]

        # Get the indices of the top-10 predicted ratings
        top_10_predicted_indices = np.argsort(user_predicted_ratings)[-1:]

        # Get the indices of the user's real ratings
        real_rated_indices = np.where(~np.isnan(user_real_ratings))[0]

        # Calculate the number of relevant items in the top-10 predicted items
        relevant_items_count = np.sum(np.isin(top_10_predicted_indices, real_rated_indices))

        # Calculate the precision for the current user
        user_precision = relevant_items_count 

        # Update the precision sum
        precision_sum += user_precision

    # Calculate the average precision across all users
    average_precision = precision_sum / users_count

    return average_precision
def precision_at_k(predicted_ratings, real_ratings, k=10):
    precision_sum = 0
    users_count = real_ratings.shape[0]

    for user_idx in range(users_count):
        user_real_ratings = real_ratings[user_idx]
        user_predicted_ratings = predicted_ratings[user_idx]

        # Get the indices of the top-k predicted ratings
        top_k_predicted_indices = np.argsort(user_predicted_ratings)[-k:]

        # Get the indices of the user's real ratings
        real_rated_indices = np.where(~np.isnan(user_real_ratings))[0]

        # Calculate the number of relevant items in the top-k predicted items
        relevant_items_count = np.sum(np.isin(top_k_predicted_indices, real_rated_indices))

        # Calculate the precision for the current user
        user_precision = relevant_items_count / k

        # Update the precision sum
        precision_sum += user_precision

    # Calculate the average precision across all users
    average_precision = precision_sum / users_count

    return average_precision

In [16]:


def k_fold_cross_validation(ratings, k=5, random_state=None):
    # Create a KFold object with the specified number of folds
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)

    # Initialize the list to store the train and test matrices for each fold
    train_test_matrices = []

    # Iterate over the splits
    for train_indices, test_indices in kf.split(ratings):
        # Copy the original ratings matrix for both train and test matrices
        train_matrix = ratings.copy()
        test_matrix = np.empty_like(ratings)

        test_matrix[:] = np.nan

        # Replace the test ratings with NaN in the train matrix and vice versa
        for row, col in zip(*np.nonzero(ratings)):
            if row in test_indices:
                train_matrix[row, col] = np.nan
                test_matrix[row, col] = ratings[row, col]

        # Add the train and test matrices to the list
        train_test_matrices.append((train_matrix, test_matrix))

    return train_test_matrices

import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

class MultiMF:
    def __init__(self, R_, P_, Q_, U_, V_, E_, D_, C_u_, C_i_, L_U_, L_V_, lambdas_, b_u_, b_v_, alpha):
        self.R_matrix = torch.tensor(R_, dtype=torch.float32)
        self.P_matrix = torch.tensor(P_, dtype=torch.float32)
        self.Q_matrix = torch.tensor(Q_, dtype=torch.float32)
        self.lambdas = lambdas_
        self.U_matrix = torch.tensor(U_, dtype=torch.float32, requires_grad=True)
        self.V_matrix = torch.tensor(V_, dtype=torch.float32, requires_grad=True)
        self.E_matrix = torch.tensor(E_, dtype=torch.float32, requires_grad=True)
        self.D_matrix = torch.tensor(D_, dtype=torch.float32, requires_grad=True)
        self.C_umatrix = torch.tensor(C_u_, dtype=torch.float32)
        self.C_imatrix = torch.tensor(C_i_, dtype=torch.float32)
        self.L_U = torch.tensor(L_U_, dtype=torch.float32)
        self.L_V = torch.tensor(L_V_, dtype=torch.float32)
        self.item_bias = torch.tensor(b_v_, dtype=torch.float32)
        self.user_bias = torch.tensor(b_u_, dtype=torch.float32)
        self.alpha = alpha
        self.total_loss = [0]

    def factorize(self, iter=200):
        self.run_func()
        for k in tqdm(range(iter)):
            if abs(self.total_loss[-1] - self.total_loss[-2]) < 0.0004:
                print("Success")
                break
            else:
                self.run_func()

    def run_func(self):
        loss_col = 0
        lambdas = self.lambdas


        for i in range(len(self.R_matrix)):  # 70
            loss_row = 0
            U = self.U_matrix[i]
            C_u = self.C_umatrix[i, :]
            C_i = self.C_imatrix[i, :]
            b_u = self.user_bias[i]
            for j in range(len(self.R_matrix[i])):  # 97

                if not torch.isnan(self.R_matrix[i, j]):
                    optimizer.zero_grad()

                    R = self.R_matrix[i, j]
                    V = self.V_matrix[:, j]
                    # E = self.E_matrix[i]
                    # Q = self.Q_matrix[j]
                    # P = self.P_matrix[i]
                    # D = self.D_matrix[j]
                    # Li = self.L_V[j]
                    # Lu = self.L_U[i]
                    E = 0
                    Q = 0
                    P = 0
                    D = 0
                    Li = 0
                    Lu = 0
                    b_v = self.item_bias[j]

                    loss = self.objective_function(R, P, Q, U, V, E, D, C_u, C_i, Lu, Li, lambdas, b_u, b_v)

                    loss.backward()
                    optimizer.step()

                    loss_row += loss.item()

            loss_col += loss_row

        self.total_loss.append(loss_col)

    def objective_function(self, R, P, Q, U, V, E, D, C_u, C_i, L_U, L_V, lambdas, b_u, b_v):
        M = U.shape[0]  # number of users
        V_ = 0
        U_ = 0

        U_ += torch.sum(C_u) * U
        V_ += torch.sum(C_i) * V
        U_mean = U_ / M
        V_mean = V_ / M
        R_pred = torch.dot(U_mean + U, V.T + V_mean)
        if P == 0:
            P_pred = torch.zeros_like(self.P_matrix)
        if Q == 0:
            Q_pred = torch.zeros_like(self.Q_matrix)
        if E == 0:
            E = torch.zeros_like(self.E_matrix)
        if D == 0:
            D = torch.zeros_like(self.D_matrix)
        loss = torch.sum((R - R_pred) ** 2) + torch.sum((P - P_pred) ** 2) + torch.sum((Q - Q_pred) ** 2)
        loss += lambdas[0] * torch.sum(U ** 2) + lambdas[1] * torch.sum(V ** 2) + lambdas[2] * torch.sum(D ** 2) + lambdas[3] * torch.sum(E ** 2)
        loss += b_u + b_v

        return loss

    def predict(self):
        self.factorize()
        mmscaler = MinMaxScaler(feature_range=(1, 6))
        R_pred = torch.matmul(self.U_matrix, self.V_matrix)
        R_pred = R_pred.detach().numpy()
        R_pred = mmscaler.fit_transform(R_pred).astype(int)
        return R_pred

    def evaluation(self, test):
        import datetime

        R_pred = self.predict()
        print(R_pred)
        mae_value = mae(R_pred, test)
        rmse_value = rmse(R_pred, test)
        top_10_f1_score_value = top_10_f1_score(R_pred, test)
        top_10_precision_value = top_10_precision(R_pred, test)
        top_1_precision_value = top_1_precision(R_pred, test)
        precision_at_k_value = precision_at_k(R_pred, test)
        result = pd.DataFrame()
        result['RMSE'] = [rmse_value]
        result['MAE'] = [mae_value]
        result['top_10_f1_score'] = [top_10_f1_score_value]
        result['top_10_precision'] = [top_10_precision_value]
        result['top_1_precision'] = [top_1_precision_value]
        result['precision_at_k'] = [precision_at_k_value]
        print(result)
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        filename = f"output_{timestamp}.csv"
        # result.to_csv(filename, index=False)


In [17]:
# df_onehot.values

In [18]:

zxz = k_fold_cross_validation(rating_matrix.replace(to_replace=0, value=np.nan).values)
train_m=zxz[0][0]
test_m =zxz[0][1]
train_m

array([[ 3.,  2.,  4., ..., nan, nan, nan],
       [ 1.,  1.,  2., ..., nan, nan, nan],
       [ 1., nan,  2., ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [19]:
test_m

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [20]:

values = [1,  100,0.1]
combinations = list(itertools.product(values, repeat=4))
combinations_arr = np.array(combinations)
selected_combinations = random.sample(combinations, 20)
print(len(combinations_arr))
model = NMF(n_components=20, init='random', random_state=0) #n_component = KNN values
U_matrix = model.fit_transform(np.nan_to_num(train_m,0))
V_matrix = model.components_
# filter_D = model.fit_transform(df_onehot.values)
# D_matrix = model.components_
# 

81


In [21]:
# L_i=np.array(calculate_L(similarity_matrix))/10

In [22]:
param_grid = {'alpha': [0.01],
              'lambdas_': selected_combinations,
              'R_': [train_m],
              'P_': [0],
              'U_':[U_matrix],
              'V_':[V_matrix],
              'Q_': [0],
              'E_': [0],
              'D_': [0],
              'C_u_': [C_umatrix],
              'C_i_': [C_imatrix],
              'L_U_': [0],
              'L_V_': [0],
              'b_u_': [user_bias.values],
              'b_v_': [item_bias.values]}
best_params = grid_search(MultiMF,param_grid, test_m)
# alpha = best_params['alpha']
# lambdas = best_params['lambdas_']
# object = MultiMF(train_m,0,0,U_matrix,V_matrix,0,0,C_umatrix,C_imatrix,0,0,lambdas,user_bias.values,item_bias.values,alpha)
# object.evaluation(test_m)

  1%|          | 2/200 [03:00<4:47:30, 87.12s/it] 

In [ ]:
object = MultiMF(train_m,0,0,U_matrix,V_matrix.T,0,0,C_umatrix,C_imatrix,0,0,[0.1,0.01,0.1,0.001,0.01],user_bias.values,item_bias.values,0.01)
# # object.factorize(5)
object.evaluation(test_m)


100%|██████████| 200/200 [00:00<00:00, 288.47it/s]

[[6 5 6 ... 4 4 4]
 [6 5 6 ... 6 5 5]
 [6 6 6 ... 6 6 6]
 ...
 [6 5 6 ... 6 5 5]
 [3 3 3 ... 3 3 3]
 [6 6 6 ... 6 6 6]]
       RMSE       MAE  top_10_f1_score  top_10_precision  top_1_precision  \
0  2.124889  1.672176         0.002008          0.001846              0.0   

   precision_at_k  
0        0.001846  


In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(object.total_loss[1:])
R_pred=object.predict()

100%|██████████| 200/200 [00:00<00:00, 318.71it/s]


In [ ]:
import matplotlib.pyplot as plt 

object.total_loss[1].item()

482270248960.0

In [ ]:
top_10_f1_score(R_pred,test_m)

0.0010658622423328306

In [ ]:
test_m.sum()

nan

In [ ]:
mmscaler = MinMaxScaler(feature_range=(1, 6))
R_pred = torch.matmul(object.U_matrix.detach(), object.V_matrix.detach().T).numpy()
R_pred = mmscaler.fit_transform(R_pred).astype(int)

In [ ]:
R_pred

array([[-2147483648, -2147483648, -2147483648, ..., -2147483648,
        -2147483648, -2147483648],
       [-2147483648, -2147483648, -2147483648, ..., -2147483648,
        -2147483648, -2147483648],
       [-2147483648, -2147483648, -2147483648, ..., -2147483648,
        -2147483648, -2147483648],
       ...,
       [-2147483648, -2147483648, -2147483648, ..., -2147483648,
        -2147483648, -2147483648],
       [-2147483648, -2147483648, -2147483648, ..., -2147483648,
        -2147483648, -2147483648],
       [-2147483648, -2147483648, -2147483648, ..., -2147483648,
        -2147483648, -2147483648]])